In [48]:
from ctypes import windll
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import os

# Set DPI scaling and Chrome options
user32 = windll.user32
user32.SetProcessDPIAware()

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
prefs = {"download.default_directory": os.getcwd() + "/data"}
chrome_options.add_experimental_option("prefs", prefs)

# Initialize the driver using ChromeDriverManager to download and set up the driver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.options = chrome_options


AttributeError: 'str' object has no attribute 'capabilities'

In [27]:
driver.get("https://globalink.mitacs.ca/#/student/application/projects")

In [28]:
print(driver.page_source)


<html><head>
  <meta charset="utf-8">
  <title>Mitacs Globalink Portals</title>
  <base href="/">

  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="icon" type="image/x-icon" href="favicon.ico">
  <link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet">
  <link href="https://use.fontawesome.com/releases/v5.0.6/css/all.css" rel="stylesheet">
  <!--<script src="https://cdnjs.cloudflare.com/ajax/libs/jspdf/1.0.272/jspdf.debug.js"></script>-->

  <!-- Global site tag (gtag.js) - Google Analytics -->
  <script type="text/javascript" async="" src="https://www.google-analytics.com/analytics.js"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-SCVM9FS164&amp;l=dataLayer&amp;cx=c"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-77075012-6"></script>
  <script>
    window.dataLayer = window.dataLayer || [];

    function gtag() {
      dataLayer.push(ar

In [29]:
import time
def nextPage():
    elements = driver.find_elements_by_xpath('//*[@id="ui-panel-1-content"]/div/p-dataview/div/p-paginator[1]/div/a[3]')[0]
    # elements.click()
    driver.execute_script("arguments[0].click();", elements)

In [38]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def execone():
    wait = WebDriverWait(driver, 10)  # Wait for a maximum of 10 seconds
    
    try:
        elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'ui-g-12') and contains(@class, 'ui-md-12') and contains(@class, 'ng-star-inserted')]")))
        #elements = driver.find_element(By.XPATH, "//div[contains(@class, 'ui-g-12') and contains(@class, 'ui-md-12') and contains(@class, 'ng-star-inserted')]")
        curPID = 0
        
        with open('output.csv', 'w') as writeFile:
            for element in elements:
                idName = element.find_elements(By.CLASS_NAME, 'ui-grid-row')
                header = idName[0].text.split('\n')
                pid = header[0].split()[-1]
                curPID = pid
                description = idName[1].find_element(By.CLASS_NAME, 'ui-grid-col-9').text

                name = description.split('\n')[0]
                description = description.split('\n')[1].replace(',', ';')
                hostDetails = element.find_element(By.CLASS_NAME, 'ui-grid-col-3').text.split('\n')
                prof = hostDetails[0].split(':')[-1].strip().replace(',', ';')
                hostProvince = hostDetails[1].split(':')[-1].strip().replace(',', ';')
                hostUniv = hostDetails[2].split(':')[-1].strip().replace(',', ';')
                language = hostDetails[3].split(':')[-1].strip().replace(',', ';')
                startDate = hostDetails[4].split(':')[-1].strip().split('(')[0].strip().replace(',', ';')
                
                writeFile.write(pid + ',' + name + ',' + description + ',' + prof + ',' + hostProvince + ',' + hostUniv + ',' + language + ',' + startDate + '\n')

        print(len(elements), curPID)
    except Exception as e:
        print("An error occurred:", str(e))
    finally:
        driver.quit()

execone()


An error occurred: Message: 



In [39]:
writeFile = open('mitacs_projects.csv', 'w', encoding="utf-8")

# Header
writeFile.write('Project ID' + ',' + 'Name' +',' + 'Description' + ',' + 'Faculty supervisor' + ',' + 'Host Province' + ',' + 'Host Institution' + ',' + 'Language' + ',' + 'Preferred start date' + '\n')

# element = elements[0]
s = 0
while s <=301:
    s += 1
    # elements = driver.find_elements_by_xpath("//div[contains(@class, 'ui-blockui') and contains(@class, 'ui-widget-overlay')]")
    # print(len(elements))
    a = 1
    while a > 0:
        a = len(driver.find_elements(By.ID, "waitBox"))
        #a=len(driver.find_element(ID, "id"))
        time.sleep(0.1)
        print('.',end = '')
    print()
    execone()
    # except:
    #     time.sleep(2)
    #     print('Failed, retrying')
    #     execone()
    # time.sleep(4)

writeFile.close()

MaxRetryError: HTTPConnectionPool(host='localhost', port=53474): Max retries exceeded with url: /session/3307635e9a540f0938f40b17c5cbae3f/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001FE446025F0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from webdriver_manager.firefox import GeckoDriverManager
import os
import time
from selenium.webdriver.chrome.service import Service

# Set Firefox options
firefox_options = FirefoxOptions()
firefox_options.add_argument("-headless")
firefox_options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"

# Get the path to the GeckoDriver executable
gecko_driver_path = GeckoDriverManager().install()
service = Service(executable_path=gecko_driver_path)
# Initialize the driver using the GeckoDriver executable
driver = webdriver.Firefox(service=service, options=firefox_options)

# Define the next_page function
def next_page():
    elements = driver.find_elements(By.CSS_SELECTOR,"#ui-panel-1-content div.p-paginator a.ui-paginator-next")
    if elements:
        elements[0].click()
        return True
    return False

# Define the execone function
def execone():
    elements = driver.find_elements(By.CSS_SELECTOR,".ui-g-12.ui-md-12.ng-star-inserted")
    soup = bs4(driver.page_source, "html.parser")
    
    with open("output.csv", "w", encoding="utf-8") as writeFile:
        for element in elements:
            data_line = extract_data(element)
            writeFile.write(data_line)
    
    return len(elements), soup

def extract_data(element):
    id_name = element.find_element_by_css_selector(".ui-grid-row").text.split("\n")
    pid = id_name[0].split()[-1]
    description = element.find_element_by_css_selector(".ui-grid-col-9").text
    name = description.split("\n")[0]
    description = description.split("\n")[1].replace(",", ";")
    host_details = element.find_element_by_css_selector(".ui-grid-col-3").text.split("\n")
    prof = host_details[0].split(":")[-1].strip().replace(",", ";")
    host_province = host_details[1].split(":")[-1].strip().replace(",", ";")
    host_univ = host_details[2].split(":")[-1].strip().replace(",", ";")
    language = host_details[3].split(":")[-1].strip().replace(",", ";")
    start_date = host_details[4].split(":")[-1].strip().split("(")[0].strip().replace(",", ";")
    return f"{pid},{name},{description},{prof},{host_province},{host_univ},{language},{start_date}\n"

# Start from the first page
driver.get("https://globalink.mitacs.ca/#/student/application/projects")

total_processed = 0
while True:
    elements_count, soup = execone()
    total_processed += elements_count
    if not next_page():
        break
    time.sleep(2)  # Optional delay between page requests

driver.quit()

print(f"Data extraction complete for {total_processed} elements.")


WebDriverException: Message: Process unexpectedly closed with status 0


In [49]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os
import time

firefox_options = FirefoxOptions()
firefox_options.add_argument("-headless")
firefox_options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"

# Get the path to the GeckoDriver executable
gecko_driver_path = r"C:\Users\ahmed\Downloads\geckodriver-v0.33.0-win64\geckodriver.exe"
service = Service(executable_path=gecko_driver_path)

# Initialize the driver using the GeckoDriver executable and Firefox options
driver = webdriver.Firefox(service=service, options=firefox_options)

driver.get("https://globalink.mitacs.ca/#/student/application/projects")


def nextPage():#p-icon p-paginator-icon
    elements = driver.find_elements(By.XPATH,'//button[@class="p-ripple p-element p-paginator-next p-paginator-element p-link"]')[0]
    print(elements)
    # elements.click()
    driver.execute_script("arguments[0].click();", elements)

def popup():
    elements = driver.find_elements(By.XPATH,'//button[@class="p-element p-button p-component"]')[0]
    print(elements)
    # elements.click()
    driver.execute_script("arguments[0].click();", elements)

def execone():
    elements = driver.find_elements(By.XPATH,"//div[contains(@class, 'col-12') and contains(@class, 'col-md-12') and contains(@class, 'ng-star-inserted')]")
    curPID = 0
    for element in elements:
        idName = element.find_elements(By.CLASS_NAME,'row')
        header = idName[0].text.split('\n')
        # name = header[-1]
        # name = name.replace(',',';')
        pid = header[0].split()[-1]
        curPID = pid
        description = idName[1].find_elements(By.CLASS_NAME,'col-9')[0].text

        name = description.split('\n')[0]
        description = description.split('\n')[1].replace(',',';')
        hostDetails = element.find_element(By.CLASS_NAME,'col-3').text.split('\n')
        # print(profs[0].text)
        prof = hostDetails[0].split(':')[-1].strip()
        prof = prof.replace(',',';')
        hostProvince = hostDetails[1].split(':')[-1].strip()
        hostProvince = hostProvince.replace(',',';')
        hostUniv = hostDetails[2].split(':')[-1].strip()
        hostUniv = hostUniv.replace(',',';')
        language = hostDetails[5].split(':')[-1].strip()
        language = language.replace(',',';')
        startDate=  hostDetails[6].split(':')[-1].strip().split('(')[0].strip()
        startDate = startDate.replace(',',';')
        popup()
        hehe=driver.find_elements(By.CLASS_NAME,'detailRow')
        hehehe=hehe[6].find_elements(By.TAG_NAME,'td')[-1].text.strip()
        writeFile.write(pid + ',' + name +',' + description + ',' + prof + ',' + hostProvince + ',' + hostUniv + ',' + language + ',' +startDate+','+ str(hehehe) + '\n')
    print(s, len(elements), curPID)
    nextPage()
# ... (Previous code remains the same)
"""
def extract_data(element):
    try:
        id_name_element = element.find_element(By.CLASS_NAME, 'ui-card-title')
        pid = id_name_element.find_element(By.TAG_NAME, 'span').text
        name = id_name_element.find_element(By.TAG_NAME, 'h3').text

        description = element.find_element(By.CLASS_NAME, 'ui-card-content').text.replace(",", ";")

        host_details = element.find_element(By.CLASS_NAME, 'ui-card-footer').text.split('\n')
        prof = host_details[1].split(': ')[1].replace(",", ";")
        host_province = host_details[2].split(': ')[1].replace(",", ";")
        host_univ = host_details[3].split(': ')[1].replace(",", ";")
        language = host_details[4].split(': ')[1].replace(",", ";")
        start_date = host_details[5].split(': ')[1].split(' (')[0].replace(",", ";")
        
        return f"{pid},{name},{description},{prof},{host_province},{host_univ},{language},{start_date}\n"
    except Exception as e:
        print("An error occurred while extracting data:", str(e))
        return ""
"""
# Start from the first page
writeFile = open('output_test.csv', 'w', encoding="utf-8")

# Header
writeFile.write('Project ID' + ',' + 'Name' +',' + 'Description' + ',' + 'Faculty supervisor' + ',' + 'Host Province' + ',' + 'Host Institution' + ',' + 'Language' + ',' + 'Preferred start date'+','+'Student academic background' + '\n')

s = 0
while s <=1:
    s += 1
    # elements = driver.find_elements_by_xpath("//div[contains(@class, 'ui-blockui') and contains(@class, 'ui-widget-overlay')]")
    # print(len(elements))
    a = 1
    while a > 0:
        a = len(driver.find_elements(By.ID,'waitBox'))
        time.sleep(0.1)
        print('.',end = '')
    print()
    execone()
writeFile.close()



"""total_processed = 0
while True:
    elements_count, soup = execone()
    total_processed += elements_count
    if not next_page():
        break
    time.sleep(2)  # Optional delay between page requests

driver.quit()"""

print(f"Data extraction complete for {total_processed} elements.")
